<a href="https://colab.research.google.com/github/a-v-kolos/Allen_dataset_neuromatch_2025/blob/anastasiia_kolos/data_extraction_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install allensdk
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.manifold import TSNE

#import brain_observatory_utilities.datasets.optical_physiology.data_formatting as ophys_formatting
#import brain_observatory_utilities.utilities.general_utilities as utilities

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

pd.set_option('display.max_columns', 500)

#ModuleNotFoundError: No module named 'brain_observatory_utilities'

In [3]:
# Создаем кеш для загрузки данных
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir="/." )

# Получаем список экспериментов
experiments = cache.get_ophys_experiment_table()

# Выбираем experiment_id (можно взять из таблицы)
experiment_id = experiments.index[0]  # первый эксперимент в таблице

# Загружаем данные эксперимента
dataset = cache.get_behavior_ophys_experiment(experiment_id)

# Получаем traces (dF/F сигналы)
traces = dataset.dff_traces

/usr/local/lib/python3.11/dist-packages/allensdk/api/cloud_cache/cloud_cache.py:439: MissingLocalManifestWarning: This cache directory appears to contain data files, but it has no record of what those files are. You might want to consider running

VisualBehaviorOphysProjectCache.construct_local_manifest()

to avoid needlessly downloading duplicates of data files that did not change between data releases. NOTE: running this method will require hashing every data file you have currently downloaded and could be very time consuming.

To avoid this warning in the future, make sure that

/_downloaded_data.json

is not deleted between instantiations of this cache
  warnings.warn(msg, MissingLocalManifestWarning)
ophys_session_table.csv: 100%|██████████| 247k/247k [00:00<00:00, 1.88MMB/s] 
behavior_session_table.csv: 100%|██████████| 1.59M/1.59M [00:00<00:00, 10.7MMB/s]
ophys_experiment_table.csv: 100%|██████████| 657k/657k [00:00<00:00, 4.82MMB/s] 
ophys_cells_table.csv: 100%|██████████| 4.28

In [4]:
experiments.head()

,behavior_session_id,ophys_session_id,ophys_container_id,mouse_id,indicator,full_genotype,driver_line,cre_line,reporter_line,sex,age_in_days,imaging_depth,targeted_structure,targeted_imaging_depth,imaging_plane_group,project_code,session_type,session_number,image_set,behavior_type,passive,experience_level,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,date_of_acquisition,equipment_name,published_at,isi_experiment_id,file_id
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951980471,951520319,951410079,1018028342,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,150,VISp,150,0,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,0
951980473,951520319,951410079,1018028345,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,225,VISp,225,0,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,1
951980475,951520319,951410079,1018028339,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,75,VISp,75,1,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,2
951980479,951520319,951410079,1018028354,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,150,VISl,150,2,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,3
951980481,951520319,951410079,1018028357,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,225,VISl,225,2,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,4


In [5]:
traces.head()

,cell_roi_id,dff
cell_specimen_id,,
1086613265,1080743723,"[0.9365729093551636, 0.5824856162071228, 1.296..."
1086613823,1080743752,"[0.555761456489563, 0.7478731870651245, 0.4397..."
1086619526,1080743754,"[0.2593991160392761, 0.30149346590042114, 0.34..."
1086614149,1080743763,"[0.4112476706504822, 0.1977161169052124, 0.143..."
1086614351,1080743765,"[0.08977238088846207, 0.0746675506234169, 0.26..."


Начнем с мыши 470784

Загрузим все сессии для мыши 470784 из experiments

In [6]:
experiments.index[0]

951980471

Что такое ophys_experiment_id?

In [7]:
cache.get_behavior_ophys_experiment(ophys_experiment_id = 951980471)

/usr/local/lib/python3.11/dist-packages/hdmf/spec/namespace.py:583: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.6.0-alpha, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)


Теперь нужно еще раз посмотреть на маленький датасет и выбрать оттуда одну мышь, посмотреть совпадет ли результат, и если да, то написать функцию с циклом.

И посмотреть потом другие условия сортировки.

In [8]:
index_85 = cache.get_behavior_ophys_experiment(ophys_experiment_id = 994278281)

behavior_ophys_experiment_994278281.nwb: 100%|██████████| 313M/313M [00:09<00:00, 33.2MMB/s]
/usr/local/lib/python3.11/dist-packages/hdmf/spec/namespace.py:583: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.6.0-alpha, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)


In [9]:
cache.get_ophys_experiment_table()

,behavior_session_id,ophys_session_id,ophys_container_id,mouse_id,indicator,full_genotype,driver_line,cre_line,reporter_line,sex,age_in_days,imaging_depth,targeted_structure,targeted_imaging_depth,imaging_plane_group,project_code,session_type,session_number,image_set,behavior_type,passive,experience_level,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,date_of_acquisition,equipment_name,published_at,isi_experiment_id,file_id
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951980471,951520319,951410079,1018028342,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,150,VISp,150,0,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,0
951980473,951520319,951410079,1018028345,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,225,VISp,225,0,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,1
951980475,951520319,951410079,1018028339,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,75,VISp,75,1,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,2
951980479,951520319,951410079,1018028354,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,150,VISl,150,2,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,3
951980481,951520319,951410079,1018028357,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,206,225,VISl,225,2,VisualBehaviorMultiscope,OPHYS_1_images_A,1,A,active_behavior,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993590480,993537991,993512923,991913246,489066,GCaMP6f,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Vip-IRES-Cre],Vip-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,128,175,VISp,175,<NA>,VisualBehaviorTask1B,OPHYS_2_images_B_passive,2,B,passive_viewing,True,Familiar,0,12,2,2019-12-10 12:25:49.549000+00:00,CAM2P.4,2021-03-25,967103425,1917
993862120,993691759,993668836,991913246,489066,GCaMP6f,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Vip-IRES-Cre],Vip-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,129,175,VISp,175,<NA>,VisualBehaviorTask1B,OPHYS_4_images_A,4,A,active_behavior,False,Novel 1,0,0,3,2019-12-11 08:57:44.877000+00:00,CAM2P.4,2021-03-25,967103425,1918
994053909,993959724,993948045,991913246,489066,GCaMP6f,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Vip-IRES-Cre],Vip-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,130,175,VISp,175,<NA>,VisualBehaviorTask1B,OPHYS_5_images_A_passive,5,A,passive_viewing,True,Novel >1,0,1,4,2019-12-12 09:29:20.785000+00:00,CAM2P.4,2021-03-25,967103425,1919


In [10]:
index_85.eye_tracking

,timestamps,cr_area,eye_area,pupil_area,likely_blink,pupil_area_raw,cr_area_raw,eye_area_raw,cr_center_x,cr_center_y,cr_width,cr_height,cr_phi,eye_center_x,eye_center_y,eye_width,eye_height,eye_phi,pupil_center_x,pupil_center_y,pupil_width,pupil_height,pupil_phi
frame,,,,,,,,,,,,,,,,,,,,,,,
0,0.37209,140.847391,67495.475720,15170.863357,False,15170.863357,140.847391,67495.475720,323.667788,325.850281,5.683365,7.888482,0.311715,343.045475,308.223660,160.163675,134.140760,-0.089899,321.523907,316.833534,65.090696,69.491264,-0.301629
1,0.38237,142.774751,67795.884093,15603.757400,False,15603.757400,142.774751,67795.884093,323.775806,324.296923,5.819092,7.809915,0.341132,343.785835,308.251036,160.782403,134.219291,-0.093592,320.962085,314.777972,64.747116,70.475742,-0.017751
2,0.38714,143.946381,67377.889962,16009.471184,False,16009.471184,143.946381,67377.889962,324.586499,325.144954,6.134908,7.468663,0.383407,344.142811,308.510901,160.148285,133.919939,-0.093933,321.204512,315.691988,66.065022,71.386084,-0.149651
3,0.42064,151.929212,66921.148605,15856.897584,False,15856.897584,151.929212,66921.148605,324.067674,325.679200,6.358265,7.605938,0.443414,343.493298,308.702761,159.985462,133.147493,-0.104477,321.578053,316.542418,66.241724,71.045107,-0.092668
4,0.45409,NaN,NaN,NaN,True,15983.102863,140.960921,67881.010384,324.151246,323.979645,5.891982,7.615308,0.324863,343.859177,306.932906,NaN,NaN,NaN,321.197549,314.719466,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136000,4533.60697,NaN,NaN,NaN,True,31769.618511,93.882190,64196.552929,325.185995,316.042488,5.188001,5.760144,-0.480468,340.950252,302.765102,NaN,NaN,NaN,335.459320,308.420243,NaN,NaN,NaN
136001,4533.63823,NaN,NaN,NaN,True,32027.457273,109.060863,64683.339091,325.548166,315.637564,5.791112,5.994557,0.741780,341.284667,302.785867,NaN,NaN,NaN,333.852640,308.700430,NaN,NaN,NaN
136002,4533.67172,NaN,NaN,NaN,True,31381.978487,98.000349,65081.291024,326.187558,315.818710,5.859079,5.324127,-0.327992,341.560411,303.003899,NaN,NaN,NaN,335.243262,302.440505,NaN,NaN,NaN


In [11]:
0.1227215975522995-0.37209

-0.24936840244770048

In [14]:
index_85.dff_traces

,cell_roi_id,dff
cell_specimen_id,,
1086496689,1080691436,"[0.1227215975522995, -0.03220267966389656, 0.1..."
1086501573,1080691448,"[0.21577566862106323, 0.26856759190559387, 0.0..."
1086500533,1080691462,"[0.20203185081481934, 0.24988459050655365, 0.2..."
1086497438,1080691464,"[0.2895065248012543, 0.010245929472148418, 0.1..."
1086497908,1080691486,"[0.31947633624076843, 0.07463130354881287, 0.2..."
1086501720,1080691501,"[0.446819543838501, 0.4650295376777649, 0.2306..."
1086497129,1080691509,"[0.22304809093475342, 0.18552088737487793, 0.1..."
1086497034,1080691515,"[0.10704667121171951, 0.0, 0.24148842692375183..."
1086496500,1080691522,"[0.4812394976615906, 0.23438869416713715, 0.08..."


In [ ]:
# Загрузите необходимые таблицы
experiments_table = cache.get_ophys_experiment_table
cells_table = cache.get_ophys_cells_table
ophys_sessions_table = cache.get_ophys_session_table()
behavior_sessions_table = cache.get_behavior_session_table()

# Получите данные о событиях стимула и ответах клеток
# Вам нужно будет выбрать конкретный эксперимент (ophys_experiment_id)
# Например, для первого эксперимента в таблице:
exp_id = experiments_table.index[0]
dataset = cache.get_behavior_ophys_experiment(exp_id)

# Получите таблицу с ответами клеток
cell_responses = dataset.get_cell_specimen_table()

# Получите данные о стимулах
stimulus_table = dataset.stimulus_presentations

# Создайте DataFrame, аналогичный вашему примеру
# Это потребует объединения данных из нескольких таблиц
df = pd.DataFrame({
    'stimulus_presentations_id': stimulus_table.index,
    'cell_specimen_id': cell_responses.index[0],  # Пример для одной клетки
    'trace': [np.array([...])],  # Здесь должны быть фактические данные трейса
    'trace_timestamps': [np.array([...])],  # Таймстемпы для трейса
    'mean_response': stimulus_table['mean_response'],
    'baseline_response': stimulus_table['baseline_response'],
    'image_name': stimulus_table['image_name'],
    'image_index': stimulus_table['image_index'],
    'is_change': stimulus_table['is_change'],
    'omitted': stimulus_table['omitted'],
    'mean_running_speed': stimulus_table['mean_running_speed'],
    'mean_pupil_area': stimulus_table['mean_pupil_area'],
    'response_latency': None,  # Эти данные могут быть в другой таблице
    'rewarded': stimulus_table['rewarded'],
    'ophys_experiment_id': exp_id,
    # Добавьте остальные поля из вашего примера, объединяя данные из разных таблиц
})

# Объедините с метаданными экспериментов
df = df.merge(experiments_table, on='ophys_experiment_id', how='left')

# Объедините с метаданными клеток
df = df.merge(cells_table, on='cell_specimen_id', how='left')

# Выберите только нужные колонки, как в вашем примере
final_columns = [
    'stimulus_presentations_id', 'cell_specimen_id', 'trace', 'trace_timestamps',
    'mean_response', 'baseline_response', 'image_name', 'image_index',
    'is_change', 'omitted', 'mean_running_speed', 'mean_pupil_area',
    'response_latency', 'rewarded', 'ophys_experiment_id', 'imaging_depth',
    'targeted_structure', 'cre_line', 'session_type', 'session_number',
    'mouse_id', 'ophys_session_id', 'ophys_container_id', 'behavior_session_id',
    'full_genotype', 'reporter_line', 'driver_line', 'indicator', 'sex',
    'age_in_days', 'exposure_level'
]

df = df[final_columns]

AttributeError: 'function' object has no attribute 'index'